In [2]:
# ultralytics instalation and cv2 instalation
%pip install ultralytics
import os
from ultralytics import YOLO
import cv2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


/Users/maxkucher/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
video_path = "path/to/your/video/in/mp4/format"
# path to output 
video_path_out = '{}_out.mp4'.format(video_path)

# this string should read file 
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
# height and width of each frame 
H, W, _ = frame.shape
# this one need to save output results
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'avc1'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

In [ ]:
# initialize path to model trained on own dataset 
model_path = "initialize/path/to/last.pt"

# Load a model
model = YOLO(model_path)  # load a custom model
# initialize threshold
threshold = 0.5

while ret:

    # put frame (bunch of frames) into model and initialize that we are going to work with object #1 (with index 0)
    results = model(frame)[0]

    
    for result in results.boxes.data.tolist():
        # initialize:
        #  - coordinates of the upper left corner (x1, y1)
        #  - coordinates of the lower right corner (x2, y2)
        #  - confidence assessment (score)
        #  - class identificator (class_id)
        # and put them all into result object 
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)

            # draw a box around detected object
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            # draw a dot in the middle of detected object
            cv2.circle(frame, (x_center, y_center), 5, (0, 255, 0), thickness=cv2.FILLED)
            # put text under the box
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()